Dato patrocinado por: FABIAN CASTRO

# Instrucciones para poder obtener tweets antiguos usando la API

Documentación: https://docs.tweepy.org/en/latest/api.html

### Paso 1: Actualizar `tweepy`

Previamente, obtener tweets antiguos era sólo posible a través de la API Premium de Twitter ($$$).

A partir de la versión 3.10.0 de `tweepy`, ahora es posible acceder a esta función de manera gratuita. 

Dado que esta nueva actualización aún está en versión alfa, para instalarla se debe usar el siguiente comando desde la terminal:

`pip install --upgrade git+https://github.com/tweepy/tweepy@master`

In [1]:
import tweepy

In [2]:
print(tweepy.__version__)

4.0.0-alpha


###  Paso 2: Crear un entorno en Twitter

Dirigirse a https://developer.twitter.com/en/account/environments y seguir los siguientes pasos.

1. Hacer click en `set up dev environment` en la sección de `full archive`
<img src='../img/clase12/tw1.png'>

2. Darle un nombre al entorno y escoger una de las aplicaciones
<img src='../img/clase12/tw2.png'>

3. Recordar el nombre del ambiente (en este ejemplo `Historico`) y saber cuál aplicación se usó para acceder con esas llaves
<img src='../img/clase12/tw3.png'>

### Ejemplo de uso

In [3]:
import pandas as pd

In [4]:
# Leer llaves
keys = pd.read_csv("../archivos/no/keys.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [5]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
# Nótese que la cadena de búsqueda ahora debe incluir otros paramétros que no son soportados en la librería
busqueda = 'Dragon Ball lang:es -"RT @"'

params = dict(label = 'Historico', # Nombre del entorno que creamos
              query = busqueda,
              fromDate = '202001010000',
              toDate = '202001020000')

In [7]:
#Cursor
tweets = tweepy.Cursor(api.search_full_archive, **params).items(200)

In [8]:
t_processed = []

for tweet in tweets:
    temp = []
    
    # Gracias Fabian
    if tweet.truncated:
        temp.append(tweet.extended_tweet['full_text'])
    else:
        temp.append(tweet.text)
        
    temp.append(tweet.created_at)
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t_processed.append(temp)

In [9]:
data = pd.DataFrame(t_processed)
data.columns = ['tweet', 'date', 'user_location', 'id', 'link']
data.head()

,tweet,date,user_location,id,link
0,Los juegos licencia son el kebab de las 4 de l...,2020-01-01 23:47:17+00:00,"Vigo, España",1212520699920822272,https://twitter.com/i/web/status/1212520699920...
1,"Bien, estoy cansado. Las Armys se la pasan dom...",2020-01-01 23:42:37+00:00,Tu pinche madre,1212519527231025152,https://twitter.com/i/web/status/1212519527231...
2,@MrTacoLz @RmArturo Y seremos para siempre Dra...,2020-01-01 23:37:28+00:00,Cdmx,1212518232394850304,https://twitter.com/i/web/status/1212518232394...
3,@NetflixLAT Por favor dame una esperanza ... D...,2020-01-01 23:37:15+00:00,None,1212518177642549248,https://twitter.com/i/web/status/1212518177642...
4,Así luce Dragon Ball Z: Kakarot con las voces ...,2020-01-01 23:36:40+00:00,España,1212518030263095301,https://twitter.com/i/web/status/1212518030263...


In [10]:
data.shape

(200, 5)

In [11]:
data.date.min()

Timestamp('2020-01-01 12:55:15+0000', tz='UTC')

In [12]:
data.date.max()

Timestamp('2020-01-01 23:47:17+0000', tz='UTC')